#### Shift Optimization
##### we will use Google OR Tools for our case. OR-Tools is an open source software suite for optimization, tuned for tackling the world's toughest problems in vehicle routing, flows, integer and linear programming, and constraint programming. Let's start with installation and imports

In [ ]:
!pip install ortools
from ortools.sat.python import cp_model
import pandas as pd

##### **Constraint Programming (CP) :** CP is the name given to identifying feasible solutions out of a very large set of candidates, where the problem can be modeled in terms of arbitrary constraints.
CP is based on feasibility (finding a feasible solution) rather than optimization (finding an optimal solution) and focuses on the constraints and variables rather than the objective function. In fact, a CP problem may not even have an objective function — the goal may simply be to narrow down a very large set of possible solutions to a more manageable subset by adding constraints to the problem.

An example of a problem that is well-suited for CP is employee scheduling.  

##### **CP-SAT solver :** To call Google's CP-SAT solver we need to call some variables, constraints and objective function.

In [ ]:
def dynamic_shift_optimization(num_employees, num_shifts, num_days, shift_density, employee_preferences):
    model = cp_model.CpModel()

    all_employees = range(num_employees)
    all_shifts = range(num_shifts)
    all_days = range(num_days)

    # Decision Variables:
    shifts = {(e, d, s): model.NewBoolVar(f"shift_e{e}_d{d}_s{s}") for e in all_employees for d in all_days for s in all_shifts}

    # Constraints :
    # 1- Shift-density constraints: Specify the number of employees required for each shift on each day
    for d in all_days:
        for s in all_shifts:
            if s == 1:  # High demand shift
                model.Add(sum(shifts[(e, d, s)] for e in all_employees) == 2)
            else:
                model.Add(sum(shifts[(e, d, s)] for e in all_employees) == shift_density)

    # 2- Employee works at most one shift per day
    for e in all_employees:
        for d in all_days:
            model.Add(sum(shifts[(e, d, s)] for s in all_shifts) <= 1)

    # Objective function :
    # Maximize Employee Satisfaction
    model.Maximize(
        sum(
            employee_preferences[e][d][s] * shifts[(e, d, s)]
            for e in all_employees
            for d in all_days
            for s in all_shifts
        )
    )

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Output Solution to DataFrame
    if status == cp_model.OPTIMAL:
        matrix_data = [[None] * num_days for _ in range(num_shifts)]

        for d in all_days:
            for s in all_shifts:
                employees_assigned = [e for e in all_employees if solver.Value(shifts[(e, d, s)]) == 1]
                matrix_data[s][d] = ', '.join(f"Employee {e}" for e in employees_assigned)

        # Creating necessary mappings
        shift_mapping = {0: "08:00-11:00", 1: "11:00-14:00", 2: "14:00-17:00"}
        matrix_df = pd.DataFrame(matrix_data, index=[f"Shift {shift_mapping[s]}" for s in all_shifts],
                                 columns=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"])

        # Calculate Total Preferences Met Metric
        total_preferences_met = sum(
            employee_preferences[e][d][s] == 1
            for e in all_employees
            for d in all_days
            for s in all_shifts
        )

        print(f"\nTotal Preferences Met: {total_preferences_met}/{num_employees * num_days * num_shifts}\n")
        print(matrix_df)

        # Save DataFrame to Excel file
        matrix_df.to_excel("shift_optimization_solution_matrix.xlsx")
        print("\nSolution saved to 'shift_optimization_solution_matrix.xlsx'")
    else:
        print("No optimal solution found!")

# Example usage
num_employees = 5
num_shifts = 3
num_days = 5  # Adjusted for 5 business days
shift_density = 1  # Example, adjust based on business needs
employee_preferences = [[[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1]],
                        [[0, 0, 0], [0, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0]],
                        [[0, 1, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0]],
                        [[0, 0, 1], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0]],
                        [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 0], [1, 0, 0]]]

dynamic_shift_optimization(num_employees, num_shifts, num_days,shift_density,employee_preferences)


Total Preferences Met: 14/75

                                   Monday                 Tuesday  \
Shift 08:00-11:00              Employee 1              Employee 0   
Shift 11:00-14:00  Employee 2, Employee 3  Employee 1, Employee 2   
Shift 14:00-17:00              Employee 0              Employee 4   

                                Wednesday                Thursday  \
Shift 08:00-11:00              Employee 3              Employee 2   
Shift 11:00-14:00  Employee 1, Employee 4  Employee 1, Employee 3   
Shift 14:00-17:00              Employee 2              Employee 4   

                                   Friday  
Shift 08:00-11:00              Employee 4  
Shift 11:00-14:00  Employee 1, Employee 2  
Shift 14:00-17:00              Employee 0  

Solution saved to 'shift_optimization_solution_matrix.xlsx'
